# Intro

Welcome to the interactive 'classification by drawing' notebook!
This notebook goes through each step and allows you to tune parameters and view how it changes the results.

The notebook proceeds as follows:

1. **Import** libraries
2. Define **paths** to data
3. Run data through the **pipeline**. (ROInet embedding + UMAP)
4. **Draw** a circle around a region of the UMAP embedding to select as 'good ROIs to keep'
5. **Visualize** results
6. **Save** results


# Import libraries

Widen the notebook

In [ ]:
# widen jupyter notebook window
from IPython.display import display, HTML
display(HTML("<style>.container {width:95% !important; }</style>"))

Import basic libraries

In [ ]:
from pathlib import Path
import tempfile

import numpy as np
from umap import UMAP

Import `roicat`

In [ ]:
%load_ext autoreload
%autoreload 2
import roicat

# Find paths to data

##### Prepare list of paths to data

In this example we are using suite2p output files, but other data types can be used (CaImAn, etc.) \
See the notebook on ingesting diverse data: https://github.com/RichieHakim/ROICaT/blob/main/notebooks/jupyter/other/demo_custom_data_importing.ipynb

Make a list containing the paths to all the input files.

In this example we are using suite2p, so the following are defined:
1. `paths_allStat`: a list to all the stat.npy files
2. `paths_allOps`: a list with ops.npy files that correspond 1-to-1 with the stat.npy files

In [ ]:
dir_allOuterFolders = r'/media/rich/bigSSD/downloads_tmp/tmp_data/mouse_0322R/statFiles/'

pathSuffixToStat = 'stat.npy'
pathSuffixToOps = 'ops.npy'

paths_allStat = roicat.helpers.find_paths(
    dir_outer=dir_allOuterFolders,
    reMatch=pathSuffixToStat,
    depth=4,
)[:1]
paths_allOps  = np.array([Path(path).resolve().parent / pathSuffixToOps for path in paths_allStat])[:]

print(f'paths to all stat files:');
[print(path) for path in paths_allStat];
print('');
print(f'paths to all ops files:');
[print(path) for path in paths_allOps];


**Important parameters**:

- `um_per_pixel` (float):
    - Resolution. 'micrometers per pixel' of the imaging field of view.

In [ ]:
data = roicat.data_importing.Data_suite2p(
    paths_statFiles=paths_allStat,
    paths_opsFiles=paths_allOps,
    um_per_pixel=2.5,  
    new_or_old_suite2p='new',
    type_meanImg='meanImgE',
    verbose=True,
)

assert data.check_completeness(verbose=False)['classification_inference'], f"Data object is missing attributes necessary for tracking."

# ROInet embedding

This step passes the images of each ROI through the ROInet neural network. The inputs are the images, the output is an array describing the visual properties of each ROI.

Initialize the ROInet object. The `ROInet_embedder` class will automatically download and load a pretrained ROInet model. If you have a GPU, this step will be much faster.

In [ ]:
DEVICE = roicat.helpers.set_device(use_GPU=True, verbose=True)
dir_temp = tempfile.gettempdir()

roinet = roicat.ROInet.ROInet_embedder(
    device=DEVICE,  ## Which torch device to use ('cpu', 'cuda', etc.)
    dir_networkFiles=dir_temp,  ## Directory to download the pretrained network to
    download_method='check_local_first',  ## Check to see if a model has already been downloaded to the location (will skip if hash matches)
    download_url='https://osf.io/c8m3b/download',  ## URL of the model
    download_hash='357a8d9b630ec79f3e015d0056a4c2d5',  ## Hash of the model file
    forward_pass_version='head',  ## How the data is passed through the network
    verbose=True,  ## Whether to print updates
)

Resize ROIs and prepare a dataloader.

**Important parameters**:
- `um_per_pixel`: (same as specified in `data` object). Resolution of FOV. This is used to resize the ROIs to be relatively consistent across resolutions.

In [ ]:
roinet.generate_dataloader(
    ROI_images=data.ROI_images,  ## Input images of ROIs
    um_per_pixel=data.um_per_pixel,  ## Resolution of FOV
    pref_plot=False,  ## Whether or not to plot the ROI sizes
    
    jit_script_transforms=False,  ## (advanced) Whether or not to use torch.jit.script to speed things up
    
    batchSize_dataloader=8,  ## (advanced) PyTorch dataloader batch_size
    pinMemory_dataloader=True,  ## (advanced) PyTorch dataloader pin_memory
    numWorkers_dataloader=-1,  ## (advanced) PyTorch dataloader num_workers. -1 is all cores.
    persistentWorkers_dataloader=True,  ## (advanced) PyTorch dataloader persistent_workers
    prefetchFactor_dataloader=2,  ## (advanced) PyTorch dataloader prefetch_factor
);

In general, you want to see that a neuron fills roughly 25-50% of the area of the image.

In [ ]:
roicat.visualization.display_toggle_image_stack(roinet.ROI_images_rs[:1000], image_size=(200,200))

Pass the data through the network. Expect for large datasets (~40,000 ROIs) that this takes around 15 minutes on CPU or 1 minute on GPU.

In [ ]:
roinet.generate_latents();

# UMAP embedding

Reduce the dimensionality of the output of ROInet (~100 dims) to 2 dimensions so that we can visualize it. Feel free to use any settings here that do a good job of clustering your data as you see fit.

In [ ]:
umap = UMAP(
    n_neighbors=25,
    n_components=2,
    n_epochs=400,
    verbose=True,
    densmap=False,
)
emb = umap.fit_transform(roinet.latents)

# Draw selection

In order to visualize the kinds of ROIs at each region of the plot, we need to select a subset of points to overlay ROI images onto.

In [ ]:
idx_images_overlay = roicat.visualization.get_spread_out_points(
    emb,
    n_ims=min(emb.shape[0], 500),
    dist_im_to_point=0.5,
    border_frac=0.05,
    device='cpu',
)

images_overlay = roinet.ROI_images_rs[idx_images_overlay]

Now we can use an interactive plot (using the holoviews library) to select our region of the scatterplot to circle.\
This plot works as follows:
- Use the **LASSO TOOL** to circle a region on the plot containing the images of ROIs that you'd like to keep/extract/mark.
    - You can circle multiple times, but only the last one will be saved
- The saved indices are saved in a temporary file that can be recovered using the `fn_get_indices` function output below. Just call `fn_get_indices()` and it will return a list of the integer indices.
- If it is difficult to see the images, do the following:
    - adjust the number of images in the above function (`roicat.visualization.get_spread_out_points`) using the `n_ims` argument
    - adjust the overlap of the images in the below function (`roicat.visualization.select_region_scatterPlot`) using the `frac_overlap_allowed` argument

In [ ]:
fn_get_indices, layout, path_tempFile = roicat.visualization.select_region_scatterPlot(
    data=emb,
    idx_images_overlay=idx_images_overlay,
    images_overlay=images_overlay[:, 6:30][:,:,6:30],
    size_images_overlay=0.6,
    frac_overlap_allowed=0.5,
    figsize=(800,800),
    alpha_points=1.0,
    size_points=10,
#     color_points=np.array(['red', 'blue', 'green', 'purple'])[labels],
    color_points='b',
);

Drop the results into easier to use output variables

In [ ]:
# roicat.helpers.export_svg_hv_bokeh(layout, '/home/rich/Desktop/umap_with_labels_dotsOnly.svg')

In [ ]:
n_sessions = len(data.ROI_images)
idx_session_cat = np.concatenate([[ii]*data.ROI_images[ii].shape[0] for ii in range(n_sessions)])
bool_good_cat = roicat.helpers.idx2bool(fn_get_indices(), length=len(idx_session_cat))
preds_good_sessions = [np.int64((bool_good_cat * (idx_session_cat==ii))[idx_session_cat==ii]) for ii in range(data.n_sessions)]

classification_output = {
    'preds': preds_good_sessions,
    'spatialFootprints': data.spatialFootprints,
    'FOV_height': data.FOV_height,
    'FOV_width': data.FOV_width,
}

# Visualize outputs

In [ ]:
print(f"Number of 'good' and 'bad' ROIs from each session:")
print([f"good: {p.sum()} / bad: {(p!=1).sum()}" for p in preds_good_sessions])

In [ ]:
FOVs_colored = roicat.visualization.compute_colored_FOV(
    spatialFootprints=data.spatialFootprints,
    FOV_height=data.FOV_height,
    FOV_width=data.FOV_width,
    labels=preds_good_sessions,
    cmap=roicat.helpers.simple_cmap([[1,0,0],[0,1,0]]),
)

roicat.visualization.display_toggle_image_stack(FOVs_colored,)

# Save results

The results file can be opened using any of the following methods:
1. `roicat.helpers.pickle_load(path)`
2. `np.load(path)`
3. ```
    import pickle
    with open(path_save, mode='rb') as f:
        test = pickle.load(f)
  ```

In [ ]:
dir_save = r'/home/rich/Desktop/'
filename_save = 'test'

path_save = str(Path(dir_save).resolve() / (filename_save + '.ROICaT.classification_drawn.results' + '.pkl'))
print(f'path_save: {path_save}')

roicat.helpers.pickle_save(classification_output, path_save)

# Thank you
If you encountered any difficulties, please let us know at the issues page: https://github.com/RichieHakim/ROICaT/issues